In [ ]:
import datetime
import numpy as np
import os
import pandas as pd
import xml.etree.ElementTree as ET

In [ ]:
file_path = "C:/users/moi/document/flux_beneficiaire.xml"
assert os.path.exists(file_path)

In [ ]:
tree = ET.parse(file_path)

In [ ]:
root = tree.getroot()

In [ ]:
ts = datetime.datetime.now()
ts_ = ts.strftime("%Y-%m-%d_%H-%M-%S")
writer = pd.ExcelWriter("extract_analyse_entrants_" + ts_ + ".xlsx")

In [ ]:
def get(node, attribute, default_value=None):
  try:
    return next(node.iter(attribute)).text
  except Exception as e:
    return default_value


In [ ]:
dossier_data = []
personne_data = []

for dossier in root.iter('InfosFoyerRSA'):
    NUMDEMRSA = get(dossier, 'NUMDEMRSA')
    ETATDOSRSA = int(get(dossier, 'ETATDOSRSA', -1))
    MOTISUSVERSRSA = get(dossier, 'MOTISUSVERSRSA', '-1')
    dossier_data.append((NUMDEMRSA, ETATDOSRSA, MOTISUSVERSRSA))

    for personne in dossier.iter('Personne'):
        personne_data.append((
            NUMDEMRSA,
            ETATDOSRSA,
            MOTISUSVERSRSA,
            get(list(personne.iter('Identification'))[0], 'NIR'),
            get(personne, 'ROLEPERS'),
            int(get(personne, 'TOPPERSDRODEVORSA', -1)),
            int(get(personne, 'TOPPERSENTDRODEVORSA', -1))
            
        ))


In [ ]:
dossier_columns = ['NUMDEMRSA', 'ETATDOSRSA', 'MOTISUSVERSRSA']
df_dossier = pd.DataFrame.from_records(dossier_data, columns=dossier_columns)
df_dossier['MOTISUSVERSRSA'] = df_dossier['MOTISUSVERSRSA'].astype('category')

In [ ]:
res_dossier = df_dossier.pivot_table(index =['ETATDOSRSA', 'MOTISUSVERSRSA'], values='NUMDEMRSA', aggfunc=len)
res_dossier.to_excel(writer, 'Dossiers')
res_dossier

In [ ]:
df_personne = pd.DataFrame.from_records(personne_data, columns=dossier_columns+['NIR', 'ROLEPERS', 'TOPPERSDRODEVORSA', 'TOPPERSENTDRODEVORSA'])
df_personne['ROLEPERS'] = df_personne['ROLEPERS'].astype('category')

In [ ]:
res_personne = df_personne[df_personne.NIR.isnull()].pivot_table(index =['ROLEPERS'], columns=['TOPPERSDRODEVORSA'],values="NIR", aggfunc=len)
res_personne.to_excel(writer, 'Personnes_sans_NIR')
res_personne

In [ ]:
res_personne_motifs = df_personne.pivot_table(index =['ETATDOSRSA', 'MOTISUSVERSRSA'], columns='ROLEPERS', values="NUMDEMRSA", aggfunc=len)
res_personne_motifs.to_excel(writer, 'Personnes_Motifs')
res_personne_motifs

In [ ]:
res_personne_ded = df_personne[df_personne.TOPPERSDRODEVORSA == 1].pivot_table(index =['ETATDOSRSA', 'MOTISUSVERSRSA'], columns='ROLEPERS', values="NUMDEMRSA", aggfunc=len)
res_personne_ded.to_excel(writer, 'Personnes_TOPPERSDRODEVORSA_1')
res_personne_ded

In [ ]:
res_p_details = df_personne.pivot_table(index =['ETATDOSRSA', 'MOTISUSVERSRSA'], columns=["TOPPERSDRODEVORSA", "ROLEPERS"], values='NUMDEMRSA', aggfunc=len)
res_p_details.to_excel(writer, 'Personnes_détails')
res_p_details

In [ ]:
REAL_ELIGIBLE_SUSPENSION_MOTIVES = (
    (df_personne.MOTISUSVERSRSA == '05') +
    (df_personne.MOTISUSVERSRSA == '44') +
    (df_personne.MOTISUSVERSRSA == '70')
)
THEORICAL_ELIGIBLE_SUSPENSION_MOTIVES = (
    (df_personne.MOTISUSVERSRSA == '01') +
    (df_personne.MOTISUSVERSRSA == '06') +
    (df_personne.MOTISUSVERSRSA == '35') +
    (df_personne.MOTISUSVERSRSA == '36')
)
ELIGIBLE_SUSPENSION_MOTIVES = REAL_ELIGIBLE_SUSPENSION_MOTIVES + THEORICAL_ELIGIBLE_SUSPENSION_MOTIVES

In [ ]:
TOP_SOUMIS_DD = (df_personne.ETATDOSRSA == 2) + (df_personne.ETATDOSRSA == 3) + (df_personne.ETATDOSRSA == 4) * ELIGIBLE_SUSPENSION_MOTIVES
df_personne['RECALC_TOP_SOUMIS_DD'] = TOP_SOUMIS_DD

In [ ]:
res_recalc = df_personne.pivot_table(index =['ETATDOSRSA', 'MOTISUSVERSRSA'], columns=["RECALC_TOP_SOUMIS_DD", "TOPPERSDRODEVORSA", "ROLEPERS"], values='NUMDEMRSA', aggfunc=len)
res_recalc.to_excel(writer, "Recalcul DeD")
res_recalc

In [ ]:
df_personne.pivot_table(index =['TOPPERSDRODEVORSA'], columns="TOPPERSENTDRODEVORSA", values='NUMDEMRSA', aggfunc=len)

In [ ]:
df_personne[df_personne.TOPPERSENTDRODEVORSA != -1].pivot_table(index =['ETATDOSRSA', 'MOTISUSVERSRSA'], columns="TOPPERSENTDRODEVORSA", values='NUMDEMRSA', aggfunc=len)

In [ ]:
writer.save()